In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Loading the data
file_path = '/content/drive/MyDrive/a5_data/'

# Define file paths
train_dir = '/content/drive/MyDrive/a5_data/train'
val_dir = '/content/drive/MyDrive/a5_data/val'

Mounted at /content/drive


#Initial Evaluations of Custom Models

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd


# Baseline transformations for data consistency
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])


# Random transformations for training data that may offer improved model performance
rand_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(30),
    transforms.ToTensor()
])


# Prepare data sets without random transformations
train_set = torchvision.datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
val_set = torchvision.datasets.ImageFolder(root=val_dir, transform=transform)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=64, shuffle=False)


# Prepare data sets with random transformations applied
train_set_rand = torchvision.datasets.ImageFolder(root=train_dir, transform=rand_transform)
train_loader_rand = torch.utils.data.DataLoader(train_set_rand, batch_size=64, shuffle=True)
val_set_rand = torchvision.datasets.ImageFolder(root=val_dir, transform=transform)
val_loader_rand = torch.utils.data.DataLoader(val_set_rand, batch_size=64, shuffle=False)


# A baseline CNN model
class BaselineCNN(nn.Module):
    def __init__(self):
        super(BaselineCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# A model with normalization
class LayerNormCNN(nn.Module):
    def __init__(self):
        super(LayerNormCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.ln1 = nn.LayerNorm([16, 64, 64]) 
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.ln2 = nn.LayerNorm([32, 32, 32]) 
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.ln3 = nn.LayerNorm([64, 16, 16]) 
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.ln1(x))

        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.relu(self.ln2(x))

        x = torch.relu(self.conv3(x))
        x = self.pool(x)
        x = torch.relu(self.ln3(x))

        x = x.view(-1, 64 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# A residual connections model
class ResConn(nn.Module):
    def __init__(self):
        super(ResConn, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=3)
        self.fc1 = nn.Linear(64 * 5 * 5, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):

        residual = self.conv1(x) 
        x = self.pool(torch.relu(residual))

        residual = self.conv2(x)
        x = self.pool(torch.relu(self.conv2(x)) + residual)

        residual = self.conv3(x)
        x = self.pool(torch.relu(self.conv3(x)) + residual)

        x = x.view(-1, 64 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


def plot_loss_accuracy(train_losses, val_losses, train_assc, val_accs, model_name):
    epochs = range(1, len(train_losses) + 1)

    # Plot loss curves
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Training Loss')
    plt.plot(epochs, val_losses, label='Validation  Loss')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot accuracy curves
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_assc, label='Training Accuracy')
    plt.plot(epochs, val_accs, label='Validation Accuracy')
    plt.title(f'{model_name} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()


def plot_confusion_matrix(model, dataloader, classes, model_name):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.numpy())
            y_pred.extend(predicted.numpy())

    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, index=classes, columns=classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_df, annot=True, cmap='Blues', fmt='d')
    plt.title(f'Confusion Matrix for {model_name}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()


def compare_models(names, accuracies):
    plt.figure(figsize=(12, 6))
    plt.bar(names, accuracies)
    plt.xticks(rotation=10, ha='right')  # Tilt the x-axis labels
    plt.title('Model Comparison')
    plt.xlabel('Model')
    plt.ylabel('Accuracy')
    plt.tight_layout()
    plt.show()


num_epochs = 200

def train_model(model, criterion, optimizer, train, val, model_name):
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    best_acc = 0
    best_epoch = 0
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        for inputs, labels in train:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_loss = running_loss / len(train)
        train_accuracy = correct_train / total_train
        train_losses.append(train_loss)
        train_accs.append(train_accuracy)

        val_loss, val_accuracy = evaluate(model, val)
        val_losses.append(val_loss)
        val_accs.append(val_accuracy)

        if val_accuracy > best_acc:
            best_acc = val_accuracy
            best_epoch = epoch + 1
            # Save the model's state dictionary
            torch.save(model.state_dict(), f'{model_name}.pth')
    print(f"Best accuracy for {model_name}, epoch: {best_epoch}:", {best_acc})
    plot_loss_accuracy(train_losses, val_losses, train_accs, val_accs, model_name)

    # Save train_losses, val_losses, train_accs, val_accs into a file
    data = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accs': train_accs,
        'val_accs': val_accs
    }
    torch.save(data, f'{model_name}_metrics.pth')

    return best_acc


def evaluate(model, dataloader):
    model.eval()     # Set the model to evaluation mode
    correct = 0
    total = 0
    total_loss = 0.0

    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    avg_loss = total_loss / len(dataloader)
    return avg_loss, accuracy


baseline_model = BaselineCNN()
baseline_model_rand = BaselineCNN()

layernorm_model = LayerNormCNN()
layernorm_model_rand = LayerNormCNN()

resconn_model = ResConn()
resconn_model_rand = ResConn()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()

# Optimizer initialization
optimizer_baseline = optim.SGD(baseline_model.parameters(), lr=0.001, momentum=0.9)
optimizer_baseline_rand = optim.SGD(baseline_model_rand.parameters(), lr=0.001, momentum=0.9)

optimizer_layernorm = optim.SGD(layernorm_model.parameters(), lr=0.001, momentum=0.9)
optimizer_layernorm_rand = optim.SGD(layernorm_model_rand.parameters(), lr=0.001, momentum=0.9)

optimizer_resconn = optim.SGD(resconn_model.parameters(), lr=0.001, momentum=0.9)
optimizer_resconn_rand = optim.SGD(resconn_model_rand.parameters(), lr=0.001, momentum=0.9)

# Train each model variant
best_acc_bs = train_model(baseline_model, criterion, optimizer_baseline, train_loader, val_loader, "Baseline Model")
best_acc_bsrt = train_model(baseline_model_rand, criterion, optimizer_baseline_rand, train_loader_rand, val_loader_rand,
                            "Baseline Model with random transformation")

best_acc_lm = train_model(layernorm_model, criterion, optimizer_layernorm, train_loader, val_loader, "Layer Normalization Model")
best_acc_lmrt = train_model(layernorm_model_rand, criterion, optimizer_layernorm_rand, train_loader_rand, val_loader_rand,
                            "Layer Normalization Model with random transformation")

best_acc_rc = train_model(resconn_model, criterion, optimizer_resconn, train_loader, val_loader, "Residual Connection Model")
best_acc_rcrt = train_model(resconn_model_rand, criterion, optimizer_resconn_rand, train_loader_rand, val_loader_rand,
                            "Residual Connection Model with random transformation")

# Define classes for confusion matrix
classes = train_set.classes

# Load trained models
baseline_model.load_state_dict(torch.load('Baseline Model.pth'))
baseline_model_rand.load_state_dict(torch.load('Baseline Model with random transformation.pth'))
layernorm_model.load_state_dict(torch.load('Layer Normalization Model.pth'))
layernorm_model_rand.load_state_dict(torch.load('Layer Normalization Model with random transformation.pth'))
resconn_model.load_state_dict(torch.load('Residual Connection Model.pth'))
resconn_model_rand.load_state_dict(torch.load('Residual Connection Model with random transformation.pth'))

# Generate confusion matrix for each model variant
plot_confusion_matrix(baseline_model, val_loader, classes, 'Baseline Model')
plot_confusion_matrix(baseline_model_rand, val_loader_rand, classes, 'Baseline Model with random transformation')
plot_confusion_matrix(layernorm_model, val_loader, classes, 'Layer Normalization Model')
plot_confusion_matrix(layernorm_model_rand, val_loader_rand, classes, 'Layer Normalization Model with random transformation')
plot_confusion_matrix(resconn_model, val_loader, classes, 'Residual Connection Model')
plot_confusion_matrix(resconn_model_rand, val_loader_rand, classes, 'Residual Connection Model with random transformation')

# Compare models based on accuracy
names = ['Baseline Model', 'Baseline Model w/ random transformation', 'Layer Normalization Model',
         'Layer Normalization Model w/ random transformation', 'Residual Connection Model',
         'Residual Connection Model w/ random transformation']
# dataloaders = [val_loader, val_loader_rand, val_loader, val_loader_rand, val_loader, val_loader_rand]
best_accuracies = [best_acc_bs, best_acc_bsrt, best_acc_lm, best_acc_lmrt, best_acc_rc, best_acc_rcrt]
compare_models(names, best_accuracies)

#ResNet18 - Pre-trained Model Approach

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
import ssl


file_path = './a5_data/'

ssl._create_default_https_context = ssl._create_unverified_context

# Defining random transformations for training data
transform_train = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Baseline transformations for validation data consistency 
transform_val = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder(file_path + 'train', transform=transform_train)
val_data = datasets.ImageFolder(file_path + 'val', transform=transform_val)

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8, shuffle=False)

# Define the Model Architecture
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # For the 2 classes: MEL and NV

# Defining Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # Learning Rate

# Training Loop
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

num_epochs = 200  # Number of training epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # 5. Validation of current epoch model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy on validation set: {100 * correct / total}%")

#Experimentation session, including saving models

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
import torch.optim as optim
import matplotlib.pyplot as plt
import csv


# Loading the data
file_path = '/content/drive/MyDrive/a5_data/'
model_path = '/content/drive/MyDrive/a5_models/'


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = downsample
        if self.downsample is None and in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(identity)

        out += identity
        out = self.relu(out)
        return out


class SimpleCNNBinary(nn.Module):
    def __init__(self):
        super(SimpleCNNBinary, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.resblock1 = ResidualBlock(32, 64, stride=2)
        self.resblock2 = ResidualBlock(64, 64)

        dummy_input = torch.zeros(1, 3, 128, 128)
        dummy_output = self.forward_features(dummy_input)
        linear_input_size = dummy_output.numel() / dummy_output.size(0)

        self.fc_layers = nn.Sequential(
            nn.Linear(int(linear_input_size), 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward_features(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.resblock1(x)
        x = self.resblock2(x)
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x


class ResidualLayerNormCNN(nn.Module):
    def __init__(self):
        super(ResidualLayerNormCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.ln1 = nn.LayerNorm([16, 64, 64]) 
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.ln2 = nn.LayerNorm([32, 32, 32])
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.ln3 = nn.LayerNorm([64, 16, 16])
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        # First convolutional block
        out = torch.relu(self.conv1(x))
        out = self.pool(out)
        out = torch.relu(self.ln1(out))

        # Second convolutional block
        out = torch.relu(self.conv2(out))
        out = self.pool(out)
        out = torch.relu(self.ln2(out))

        # Third convolutional block
        out = torch.relu(self.conv3(out))
        out = self.pool(out)
        out = torch.relu(self.ln3(out))

        # Fully connected layers
        out = out.view(-1, 64 * 16 * 16)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

# Random transformations
transform_train = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Data consistency transformations
transform_val = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder(file_path + 'train', transform=transform_train)
val_data = datasets.ImageFolder(file_path + 'val', transform=transform_val)

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8, shuffle=False)

# Preview the training images
Xexamples, Yexamples = next(iter(train_loader))

for i in range(8):
    plt.subplot(2,4,i+1)
    img = Xexamples[i].numpy().transpose(1, 2, 0)
    plt.imshow(img, interpolation='none')
    plt.title('NV' if Yexamples[i] else 'MEL')
    plt.xticks([])
    plt.yticks([])

plt.show()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ResidualLayerNormCNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 200 # Number of training epochs

# Open a file to save the epoch, loss, and accuracy data
with open('./training_metrics.txt', 'w') as f:
    f.write('Epoch,Loss,Validation Accuracy\n')

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        correct = 0
        total = 0
        with torch.no_grad():  # No gradient computation in evaluation phase
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
                outputs = model(inputs)
                predicted = torch.sigmoid(outputs) > 0.5  # Convert to binary predictions
                total += labels.size(0)
                correct += (predicted == labels).type(torch.float).sum().item()

        accuracy = 100 * correct / total

        # Save metrics to file
        f.write(f'{epoch+1},{epoch_loss:.4f},{accuracy:.2f}\n')

        print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")
        print(f"Validation Accuracy: {accuracy:.2f}%")

        # Save model checkpoint
        if (epoch > 119): # Adjustment, models from epochs 0-119 rarely offered competetive results
            torch.save(model, f'{model_path}model_wa_epoch_{epoch+1}.pth')